## Load the data set

In [ ]:
from xautoml.util.datasets import hearts

X_train, y_train = hearts('../heart.csv', train=True)
X_train

## Start the Model Building

In [ ]:
import pickle
from dswizard.optimizers.bandit_learners import PseudoBandit
from dswizard.optimizers.structure_generators.mcts import TransferLearning, MCTS
from dswizard.optimizers.config_generators import Hyperopt
from dswizard.core.master import Master
from dswizard.core.model import Dataset
from dswizard.util import util

util.setup_logging('output/mcts/log.txt')

ds = Dataset(X_train.values, y_train.values, task=-2, metric='accuracy', feature_names=X_train.columns)
master = Master(
    ds=ds,
    working_directory='output/mcts/',
    n_workers=1,
    model='rf_complete.pkl',

    wallclock_limit=300,
    cutoff=10,
    pre_sample=False,

    config_generator_class=Hyperopt,
    structure_generator_class=MCTS,
    structure_generator_kwargs={'policy': TransferLearning},
    bandit_learner_class=PseudoBandit
)

pipeline, run_history, ensemble = master.optimize()

with open('output/mcts/dswizard.pkl', 'wb') as f:
    pickle.dump((run_history, ensemble), f)


In [ ]:
import pickle

with open('output/mcts/dswizard.pkl', 'rb') as f:
    run_history, ensemble = pickle.load(f)

## Get the Score of the Final Ensemble

In [ ]:
from sklearn.metrics import accuracy_score

X_test, y_test = hearts('../heart.csv', test=True)

predictions = ensemble.predict(X_test.values)
accuracy_score(y_test, predictions)

In [ ]:
## View the Models found by dswizard

In [ ]:
from xautoml.main import XAutoML
from xautoml.adapter import import_dswizard

rh = import_dswizard(run_history, ensemble)
main = XAutoML(rh, X_test, y_test)
main